In [ ]:
!git clone https://github.com/chengjiali/Prompt-Transferability.git
%cd Prompt-Transferability
!pip install -r requirements.txt

fatal: destination path 'Prompt-Transferability' already exists and is not an empty directory.
/content/Prompt-Transferability/Prompt-Transferability-2.0-latest/Prompt-Transferability


In [ ]:
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.9.0+cu111 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.

In [ ]:
%cd Prompt-Transferability-2.0-latest/

/content/Prompt-Transferability/Prompt-Transferability-2.0-latest/Prompt-Transferability/Prompt-Transferability-2.0-latest


In [ ]:
import json
import os
import sys
import torch
import logging
import random
import numpy as np

from datasets import load_dataset, load_metric
from transformers.trainer_utils import get_last_checkpoint
from transformers import (
    set_seed,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EvalPrediction,
    default_data_collator,
)
from openprompt.data_utils.utils import InputExample
from openprompt import PromptDataLoader, PromptForClassification
from openprompt.plms import load_plm
from openprompt.prompts import SoftTemplate, ManualVerbalizer

from prompt_hub import task_to_keys, get_model
from prompt_hub.hub import PromptHub
from prompt_hub.training_args import PromptTrainingArguments, RemainArgHfArgumentParser

## Train prompt

Let's first generate a config and convert it into `huggingface.TrainingArguments`.

In [ ]:
config = {
    "do_train": True,
    "do_eval": True,
    "do_test": True,
    "prompt_len": 100,
    "num_proj_layers": 1,
    "flatten_proj": True,
    "max_steps": 100,
    "eval_steps": 10,
    "save_steps": 10,
    "logging_steps": 2,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 128,
    "learning_rate": 5e-4,
    "lr_scheduler_type": 'constant',
    "warmup_steps": 0,

    "save_total_limit": 1,
    "predict_with_generate": True,
    "load_best_model_at_end": True,
    "metric_for_best_model": "combined_score",
    "greater_is_better": True,
    "evaluation_strategy": "steps",
    "overwrite_output_dir": True,
    "save_strategy": "steps",
    "model_parallel": False
}

args = PromptTrainingArguments(
    output_dir='outputs',
    backbone='roberta-base',
    dataset='rotten_tomatoes',
    prompt_len=100
)

for k, v in config.items():
  setattr(args, k, v)

In [ ]:
set_seed(args.seed)

is_regression = args.dataset in ['stsb']

metric = load_metric("prompt_hub/glue_metrics.py", args.dataset)

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
    result = metric.compute(predictions=preds, references=p.label_ids)
    result["combined_score"] = np.mean(list(result.values())).item()

    return result

/usr/local/lib/python3.10/dist-packages/datasets/load.py:848: FutureWarning: The repository for glue_metrics contains custom code which must be executed to correctly load the metric. You can inspect the repository content at prompt_hub/glue_metrics.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
trainer = PromptHub(
    args=args,
    compute_metrics=compute_metrics,
)

Trainable parameters:
prompt_model.template.soft_embeds torch.Size([100, 768])
Template: SoftTemplate(
  (raw_embedding): Embedding(50265, 768, padding_idx=1)
)
Verbalizer: ManualVerbalizer()
Raw input example: {
  "guid": 0,
  "label": 1,
  "meta": {},
  "text_a": "the rock is destined to be the 21st century's new \" conan \" and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .",
  "text_b": "",
  "tgt_text": null
}

Wrapped input example: [[{'text': '', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': ' the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' ', 'loss_ids': 0, 'shortenable_ids': 1}], {'guid': 0, 'label': 1}]


In [ ]:
# cấu trúc input
trainer.template.text

[{'add_prefix_space': '', 'soft': None, 'duplicate': 100, 'same': True},
 {'add_prefix_space': ' ', 'mask': None},
 {'add_prefix_space': ' ', 'placeholder': 'text_a'},
 {'add_prefix_space': ' ', 'placeholder': 'text_b'}]

In [ ]:
# độ dài soft prompt
trainer.template.num_tokens

100

In [ ]:
# kích thước prompt embedding (num_tokens, embedding_dim)
trainer.template.soft_embeds.shape

torch.Size([100, 768])

In [ ]:
# mô hình
args.backbone

'roberta-base'

In [ ]:
# tác vụ
args.dataset

'rotten_tomatoes'

#### Train Prompt

To save time, we use a pretrained prompt.

In [ ]:
trainer.train_prompt(args.backbone, args.dataset)

tokenizing: 8530it [00:07, 1163.22it/s]


Step,Training Loss,Validation Loss,Accuracy,Combined Score
10,0.622200,0.656431,63.320826,63.320826
20,0.714100,0.631427,64.634146,64.634146
30,0.627700,0.611055,66.697936,66.697936
40,0.566900,0.593935,68.761726,68.761726
50,0.613400,0.573224,71.294559,71.294559
60,0.590900,0.560099,72.420263,72.420263
70,0.609800,0.550776,73.170732,73.170732
80,0.491200,0.541496,73.733583,73.733583
90,0.528800,0.530911,73.170732,73.170732
100,0.623300,0.525940,74.859287,74.859287


tokenizing: 1066it [00:03, 346.76it/s]
tokenizing: 1066it [00:00, 1803.06it/s]
tokenizing: 1066it [00:00, 1781.72it/s]
tokenizing: 1066it [00:00, 1803.07it/s]
tokenizing: 1066it [00:00, 1071.73it/s]
tokenizing: 1066it [00:00, 1698.75it/s]
tokenizing: 1066it [00:00, 1742.05it/s]
tokenizing: 1066it [00:01, 1035.03it/s]
tokenizing: 1066it [00:00, 1722.40it/s]
tokenizing: 1066it [00:00, 1734.05it/s]
There were missing keys in the checkpoint model loaded: ['prompt_model.plm.roberta.embeddings.word_embeddings.weight', 'prompt_model.plm.roberta.embeddings.position_embeddings.weight', 'prompt_model.plm.roberta.embeddings.token_type_embeddings.weight', 'prompt_model.plm.roberta.embeddings.LayerNorm.weight', 'prompt_model.plm.roberta.embeddings.LayerNorm.bias', 'prompt_model.plm.roberta.encoder.layer.0.attention.self.query.weight', 'prompt_model.plm.roberta.encoder.layer.0.attention.self.query.bias', 'prompt_model.plm.roberta.encoder.layer.0.attention.self.key.weight', 'prompt_model.plm.roberta.

TrainOutput(global_step=100, training_loss=0.5867584550380707, metrics={'train_runtime': 254.2626, 'train_samples_per_second': 6.293, 'train_steps_per_second': 0.393, 'total_flos': 0.0, 'train_loss': 0.5867584550380707, 'epoch': 0.19})

#### Evaluate Prompt

Now we can evaluate the trained prompt.

In [ ]:
eval_results = trainer.eval_prompt(args.backbone, args.dataset)
print(eval_results)

tokenizing: 1066it [00:00, 1732.50it/s]


#### Cross-Task evaluation

Now let's do a cross-task evaluation. We evaluate the prompt trained on MNLI on SNLI.

In [ ]:
cross_task_eval_results = trainer.cross_task_eval(args.backbone, 'rotten_tomatoes')
print(cross_task_eval_results)